# Malc0de Web Scraping Project

### Malc0de is a web page that stores details of urls which contains malicious files but it has blocked indian ips from opening it directly. We have to either break google captcha or use proxy network to scrape data from it. So first we are scraping data using beautifulsoup and requests. Now we will extract geo information of ips scraped and finally dump it to a csv file.

In [ ]:
import requests
from ip2geotools.databases.noncommercial import DbIpCity
import time
import csv
import pandas as pd
import sys
from itertools import cycle
import traceback
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml.html import fromstring

In [ ]:
def getproxy():                           #function to get US proxies from this website 
    url="https://www.us-proxy.org/"
    page=requests.get(url)
    parser=fromstring(page.text)
    proxies=set()
    for i in parser.xpath('//tbody/tr')[:20]:
        proxy=":".join([i.xpath('.//td[1]/text()')[0],i.xpath('.//td[2]/text()')[0]])
        proxies.add(proxy)
    return proxies

proxies=getproxy()
proxypool=cycle(proxies)
title=[]
URL=[]
IP=[]
MD5=[]
ASN=[]
status=0
link="http://malc0de.com/rss/"                    #scraping contents from this webpage using proxy
for x in range(1,11):
    proxy=next(proxypool)
    print("Requesting proxy connection ",proxy)
    try:
        page=requests.get(link,proxies={"http":proxy,"https":proxy})
        soup=BeautifulSoup(page.content,"lxml")
        m=soup.findAll("item")
        print(m)
        for y in m:
            t=y.find("title").text
            title.append(t)
            d=y.find("description").text.split()    #splitting data using space as delimeter
            URL.append(d[1].replace(',', ''))       #make list of splitted data according to its index and removing , in end
            IP.append(d[4].replace(',', ''))
            ASN.append(d[8].replace(',', ''))
            MD5.append(d[10])
            status=1
        if(status==1):
            break;
    except:
        print("Can't connect, Skipping")

dictionary={
    "Title" : title,
    "URL" : URL,
    "IP" : IP,
    "MD5" : MD5,
    "ASN" : ASN
}

# print(dictionary)
malc0de=pd.DataFrame(dictionary)              #Making more fields with default value for ip geo info
malc0de["Country"]="Unknown"
malc0de["Region"]="Unknown"
malc0de["City"]="Unknown"
malc0de["Longitude"]="Unknown"
malc0de["Latitude"]="Unknown"
malc0de["IPinfo"]="Incomplete"
malc0de["Status"]="Not Downloaded"
file=open("D:\EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv","a",encoding="utf-8") #dumping to a csv file
malc0de.to_csv(file, sep=',', index=False)
file.close()
# malc0de   
print("Operation Successfull")

In [ ]:
#Removing duplicates and updating it in csv
y=pd.read_csv('D:\EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv',encoding="utf-8") 
y.sort_values("MD5",inplace=True)
y.drop_duplicates(subset='MD5',keep="first")
x=pd.DataFrame(y)

file2=open("D:\EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv","w",encoding="utf-8")
x.to_csv(file2, sep=',',index=False)
file2.close()

In [ ]:
#remove duplicates from csv and update it
y=pd.read_csv('D:\EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv',encoding="utf-8")
y.sort_values("MD5",inplace=True)
y.drop_duplicates(subset='MD5',keep="first")
x=pd.DataFrame(y)

file2=open("D:\EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv","w",encoding="utf-8")
x.to_csv(file2, sep=',',index=False)
file2.close()
 
#Code to extract geographic details of Ip addresses scraped

iplist=[]
region=[]
country=[]
city=[]
longitude=[]
latitude=[]
IPinfo=[]

x=pd.read_csv('D:\EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv',encoding="utf-8")
ipls=x["IP"]
rels=x["Region"]
cils=x["City"]
lols=x["Longitude"]
lals=x["Latitude"]
ctls=x["Country"]
ipinfols=x["IPinfo"]

for n in ipinfols:
    IPinfo.append(n)
    
for n in ipls:
    iplist.append(n)
    
for n in rels:
    region.append(n)
    
for n in cils:
    city.append(n)
    
for n in lols:
    longitude.append(n)
    
for n in lals:
    latitude.append(n)

for n in ctls:
    country.append(n)
    
iplen=len(iplist)
print("Length= ",iplen)
print("\nIP LIST:\n",iplist)
print("*****-----*****")
print("\nCountry:\n",country)
print("*****-----*****")
print("\nCity:\n",city)
print("*****-----*****")
print("\nLongitude:\n",longitude)
print("*****-----*****")
print("\nLatitude:\n",latitude)
print("*****-----*****")
print("\nRegion:\n",region)
print("*****-----*****")
print("\nIP info:\n",IPinfo)
print("*****-----*****")
for i in range(iplen):
    print("\ni=",i)
    if(IPinfo[i]=="Incomplete"):
        try:
            try:
                f=open('D:\EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv',encoding="utf-8")
            except:
                print("Malc0de Csv File not found")
                sys.exit()
            print("\nIP: ",iplist[i])      
            r = csv.reader(f)            
            records = list(r)
            records =[e for e in records if e]                       #Remove blank spaces from record when converted to list
            response = DbIpCity.get(iplist[i], api_key='free')       #get response(geo info) when ip is passed    
#             time.sleep(5)           
            records[i+1][5]=response.country
            records[i+1][6]=response.region
            records[i+1][7]=response.city
            records[i+1][8]=response.longitude
            records[i+1][9]=response.latitude
            records[i+1][10]="Complete"                           #update status of ipinfo
            f.close()            
            f=open('D:EDUCATIONAL\CERT-In\Malc0de Project\Malc0de.csv','w',encoding="utf-8")           #update csv
            w = csv.writer(f)
            w.writerows(records) 
            f.close()          
            print("-->Details of IP: ",iplist[i]," Successfully Retrieved")            
        except:
            pass
    else:
        pass
print("operation done")